# Predicting house prices using K-Nearest neighbors regression
In this notebook, we will implement k-nearest neighbors regression. We will:
  * Find the k-nearest neighbors of a given query input
  * Predict the output for the query input using the k-nearest neighbors
  * Choose the best value of k using a validation set

# Fire up GraphLab Create

In [7]:
import graphlab
import numpy as np

# Load in house sales data

For this notebook, we use a subset of the King County housing dataset created by randomly selecting 40% of the houses in the full dataset.

In [8]:
sales = graphlab.SFrame('kc_house_data_small.gl/')

# Import useful functions from previous notebooks

To efficiently compute pairwise distances among data points, we will convert the SFrame into a 2D Numpy array.

# Converting SFrame to Numpy array

In [9]:
def get_numpy_data(data_sframe, features, output):
    data_sframe['constant'] = 1
    features = ['constant'] + features 
    features_sframe = data_sframe[features]
    feature_matrix = features_sframe.to_numpy()
    output_sarray = data_sframe[output]
    output_array = output_sarray.to_numpy()
    
    return feature_matrix, output_array

We will also need the `normalize_features()` function that normalizes all feature columns to unit norm

# Normalizing the features

In [11]:
def normalize_features(features_matrix):
    norms = np.linalg.norm(features_matrix, axis = 0)
    normalized_features = features_matrix / norms
    return normalized_features, norms

# Split data into training, test, and validation sets

In [12]:
(train_and_validation, test) = sales.random_split(.8, seed=1) # initial train/test split
(train, validation) = train_and_validation.random_split(.8, seed=1) # split training set into training and validation sets

# Extract features and normalize

Using all of the numerical inputs listed in `feature_list`, transforming the training, test, and validation SFrames into Numpy arrays:

In [13]:
feature_list = ['bedrooms',  
                'bathrooms',  
                'sqft_living',  
                'sqft_lot',  
                'floors',
                'waterfront',  
                'view',  
                'condition',  
                'grade',  
                'sqft_above',  
                'sqft_basement',
                'yr_built',  
                'yr_renovated',  
                'lat',  
                'long',  
                'sqft_living15',  
                'sqft_lot15']
features_train, output_train = get_numpy_data(train, feature_list, 'price')
features_test, output_test = get_numpy_data(test, feature_list, 'price')
features_valid, output_valid = get_numpy_data(validation, feature_list, 'price')

In computing distances, it is crucial to normalize features. Otherwise, for example, the `sqft_living` feature (typically on the order of thousands) would exert a much larger influence on distance than the `bedrooms` feature (typically on the order of ones). We divide each column of the training feature matrix by its 2-norm, so that the transformed column has unit norm.

IMPORTANT: Make sure to store the norms of the features in the training set. The features in the test and validation sets must be divided by these same norms, so that the training, test, and validation sets are normalized consistently.

In [14]:
features_train, norms = normalize_features(features_train) # normalize training set features (columns)
features_test = features_test / norms # normalize test set by training set norms
features_valid = features_valid / norms # normalize validation set by training set norms

# Compute a single distance

To start, let's just explore computing the "distance" between two given houses.  We will take our **query house** to be the first house of the test set and look at the distance between this house and the 10th house of the training set.

To see the features associated with the query house, we will print the first row (index 0) of the test feature matrix. We should get an 18-dimensional vector whose components are between 0 and 1.

Now printing the 10th row (index 9) of the training feature matrix. Again, we get an 18-dimensional vector with components between 0 and 1.

In [138]:
query_house = features_test[0]
house_10 = features_train[9]

# Calculating Euclidean distance between two houses

In [19]:
def distance_between_houses(house_1, house_2):
    difference = house_1 - house_2
    euclidean_distance = np.sqrt(np.sum(difference**2))
    return euclidean_distance

In [20]:
distance_between_houses(query_house, house_10)

0.05972359371666126

# Compute multiple distances

Of course, to do nearest neighbor regression, we need to compute the distance between our query house and *all* houses in the training set.  

To visualize this nearest-neighbor search, let's first compute the distance from our query house (`features_test[0]`) to the first 10 houses of the training set (`features_train[0:10]`) and then search for the nearest neighbor within this small set of houses.  Through restricting ourselves to a small set of houses to begin with, we can visually scan the list of 10 distances to verify that our code for finding the nearest neighbor is working.

**Writing a function to compute the Euclidean distance from the query house to each of the first 10 houses in the training set.**

In [21]:
def distance_between_multiple_houses(query_house, set_of_houses):
    euclidean_distance = []
    for house in set_of_houses:
        distance = distance_between_houses(house, query_house)
        euclidean_distance.append(distance)
        
    return euclidean_distance

**Calculating Among the first 10 training houses, which house is the closest to the query house?**

In [42]:
dist = distance_between_multiple_houses(query_house, features_train[:10])
for i, distance in enumerate(dist):
    print 'House '+str(i)+':  %0.3f'%distance

House 0:  0.060
House 1:  0.085
House 2:  0.061
House 3:  0.053
House 4:  0.058
House 5:  0.060
House 6:  0.055
House 7:  0.055
House 8:  0.052
House 9:  0.060


It is computationally inefficient to loop over computing distances to all houses in our training dataset. Fortunately, many of the Numpy functions can be **vectorized**, applying the same operation over multiple values or vectors.  We now walk through this process.

Consider the following loop that computes the element-wise difference between the features of the query house (`features_test[0]`) and the first 3 training houses (`features_train[0:3]`):

In [35]:
for i in xrange(3):
    print features_train[i]-features_test[0]
    # should print 3 vectors of length 18

[ 0.00000000e+00 -3.87821276e-03 -1.20498190e-02 -1.05552733e-02
  2.08673616e-04 -8.52950206e-03  0.00000000e+00 -5.10236549e-02
  0.00000000e+00 -3.47633726e-03 -5.50336860e-03 -2.48168183e-02
 -1.63756198e-04  0.00000000e+00 -1.70072004e-05  1.30577772e-05
 -5.14364795e-03  6.69281453e-04]
[ 0.00000000e+00 -3.87821276e-03 -4.51868214e-03 -2.26610387e-03
  7.19763456e-04  0.00000000e+00  0.00000000e+00 -5.10236549e-02
  0.00000000e+00 -3.47633726e-03  1.30705004e-03 -1.45830788e-02
 -1.91048898e-04  6.65082271e-02  4.23240653e-05  6.22415897e-06
 -2.89330197e-03  1.47606982e-03]
[ 0.00000000e+00 -7.75642553e-03 -1.20498190e-02 -1.30002801e-02
  1.60518166e-03 -8.52950206e-03  0.00000000e+00 -5.10236549e-02
  0.00000000e+00 -5.21450589e-03 -8.32384500e-03 -2.48168183e-02
 -3.13866046e-04  0.00000000e+00  4.71047219e-05  1.56530415e-05
  3.72914476e-03  1.64764925e-03]


The subtraction operator (`-`) in Numpy is vectorized as follows:

In [36]:
print features_train[0:3] - features_test[0]

[[ 0.00000000e+00 -3.87821276e-03 -1.20498190e-02 -1.05552733e-02
   2.08673616e-04 -8.52950206e-03  0.00000000e+00 -5.10236549e-02
   0.00000000e+00 -3.47633726e-03 -5.50336860e-03 -2.48168183e-02
  -1.63756198e-04  0.00000000e+00 -1.70072004e-05  1.30577772e-05
  -5.14364795e-03  6.69281453e-04]
 [ 0.00000000e+00 -3.87821276e-03 -4.51868214e-03 -2.26610387e-03
   7.19763456e-04  0.00000000e+00  0.00000000e+00 -5.10236549e-02
   0.00000000e+00 -3.47633726e-03  1.30705004e-03 -1.45830788e-02
  -1.91048898e-04  6.65082271e-02  4.23240653e-05  6.22415897e-06
  -2.89330197e-03  1.47606982e-03]
 [ 0.00000000e+00 -7.75642553e-03 -1.20498190e-02 -1.30002801e-02
   1.60518166e-03 -8.52950206e-03  0.00000000e+00 -5.10236549e-02
   0.00000000e+00 -5.21450589e-03 -8.32384500e-03 -2.48168183e-02
  -3.13866046e-04  0.00000000e+00  4.71047219e-05  1.56530415e-05
   3.72914476e-03  1.64764925e-03]]


Note that the output of this vectorized operation is identical to that of the loop above, which can be verified below:

In [37]:
# verify that vectorization works
results = features_train[0:3] - features_test[0]
print results[0] - (features_train[0]-features_test[0])
# should print all 0's if results[0] == (features_train[0]-features_test[0])
print results[1] - (features_train[1]-features_test[0])
# should print all 0's if results[1] == (features_train[1]-features_test[0])
print results[2] - (features_train[2]-features_test[0])
# should print all 0's if results[2] == (features_train[2]-features_test[0])

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


# Perform 1-nearest neighbor regression

Now that we have the element-wise differences, it is not too hard to compute the Euclidean distances between our query house and all of the training houses. First, write a single-line expression to define a variable `diff` such that `diff[i]` gives the element-wise difference between the features of the query house and the `i`-th training house.

In [38]:
diff = features_train - features_test[0]

To test the code above, run the following cell, which should output a value -0.0934339605842:

In [39]:
print diff[-1].sum() # sum of the feature differences between the query and last training house
# should print -0.0934339605842

-0.0934339605841801


The next step in computing the Euclidean distances is to take these feature-by-feature differences in `diff`, square each, and take the sum over feature indices.  That is, compute the sum of square feature differences for each training house (row in `diff`).

By default, `np.sum` sums up everything in the matrix and returns a single number. To instead sum only over a row or column, we need to specifiy the `axis` parameter described in the `np.sum` [documentation](http://docs.scipy.org/doc/numpy-1.10.1/reference/generated/numpy.sum.html). In particular, `axis=1` computes the sum across each row.

Below, we compute this sum of square feature differences for all training houses and verify that the output for the 16th house in the training set is equivalent to having examined only the 16th row of `diff` and computing the sum of squares on that row alone.

In [40]:
print np.sum(diff**2, axis=1)[15] # take sum of squares across each row, and print the 16th sum
print np.sum(diff[15]**2) # print the sum of squares for the 16th row -- should be same as above

0.003307059028786791
0.0033070590287867904


With this result in mind, we are going to write a single-line expression to compute the Euclidean distances between the query house and all houses in the training set. Then we will assign the result to a variable `distances`.

In [43]:
distances = np.sqrt(np.sum(diff**2, axis = 1))

To test the code above, we are going to run the following cell, which should output a value 0.0237082324496:

In [44]:
print distances[100] # Euclidean distance between the query house and the 101th training house
# should print 0.0237082324496

0.023708232449603735


Now we are ready to write a function that computes the distances from a query house to all training houses. The function should take two parameters: (i) the matrix of training features and (ii) the single feature vector associated with the query.

# Computing distances to all training house with query house

In [45]:
def faster_distance_between_multiple_houses(query_house, set_of_training_house):
    diff = set_of_training_house - query_house
    return np.sqrt(np.sum(diff**2, axis = 1))

In [48]:
list_of_distance = faster_distance_between_multiple_houses(features_test[2], features_train)
distance_dictionary = dict(zip(range(len(list_of_distance)), list_of_distance))

1.  Taking the query house to be third house of the test set (`features_test[2]`).  What is the index of the house in the training set that is closest to this query house?
2.  What is the predicted value of the query house based on 1-nearest neighbor regression?

In [57]:
sorted_list = sorted(distance_dictionary.items(), key = lambda x : x[1])
sorted_list[0]

(382, 0.002860495267507927)

In [55]:
output_train[382]

249000

# Perform k-nearest neighbor regression

For k-nearest neighbors, we need to find a *set* of k houses in the training set closest to a given query house. We then make predictions based on these k nearest neighbors.

## Fetch k-nearest neighbors

Using the functions above, implement a function that takes in
 * the value of k;
 * the feature matrix for the training houses; and
 * the feature vector of the query house
 
and returns the indices of the k closest training houses. For instance, with 2-nearest neighbor, a return value of [5, 10] would indicate that the 6th and 11th training houses are closest to the query house.

**Look at the [documentation for `np.argsort`](http://docs.scipy.org/doc/numpy/reference/generated/numpy.argsort.html)**

In [95]:
def k_nearest_neighbor(query_house, set_of_houses, k):
    distance = faster_distance_between_multiple_houses(query_house, set_of_houses)
    distance_dict = dict(zip(range(len(list_of_distance)), list_of_distance))
    sorted_distance = sorted(distance_dictionary.items(), key = lambda x : x[1])
    return sorted_distance[:k]

In [96]:
k_nearest_neighbor(features_test[2], features_train, 4)

[(382, 0.002860495267507927),
 (1149, 0.0032258403880432852),
 (4087, 0.003502156501575598),
 (3142, 0.0035931537790271787)]

## Make a single prediction by averaging k nearest neighbor outputs

Now that we know how to find the k-nearest neighbors, write a function that predicts the value of a given query house. **For simplicity, take the average of the prices of the k nearest neighbors in the training set**. The function should have the following parameters:
 * the value of k;
 * the feature matrix for the training houses;
 * the output values (prices) of the training houses; and
 * the feature vector of the query house, whose price we are predicting.
 
The function should return a predicted value of the query house.

**We can extract multiple items from a Numpy array using a list of indices. For instance, `output_train[[6, 10]]` returns the prices of the 7th and 11th training houses.**

# Predicting Average price of house using K-Nearest Neighbor

In [70]:
def predict_k_nearest_neighbor(k_nearest_neighbor, output_train):
    pred = 0
    for prediction, _ in k_nearest_neighbor:
        pred += output_train[prediction]
    return pred / len(k_nearest_neighbor)

In [71]:
predict_using_k_nearest_neighbor(k_nearest_neighbor(features_test[2], features_train, 4), output_train)

413987

## Making multiple predictions

We are going to write a function to predict the value of *each and every* house in a query set. (The query set can be any subset of the dataset, be it the test set or validation set.) The idea is to have a loop where we take each house in the query set as the query house and make a prediction for that specific house. The new function should take the following parameters:
 * the value of k;
 * the feature matrix for the training houses;
 * the output values (prices) of the training houses; and
 * the feature matrix for the query set.
 
The function is going to return a set of predicted values, one for each house in the query set.

# Computing distance using Numpy's Argsort function

In [97]:
def k_nearest_neighbor_argsort(query_house, set_of_houses, k):
    distance = faster_distance_between_multiple_houses(query_house, set_of_houses)
    return np.argsort(distance, axis = 0)[:k]

# Predicting on Set of Multiple Test House query

In [98]:
def final_k_nearest_neighbor(k, set_of_houses, output_train, query_house_set):
    predictions_dict = {}
    for i in range(query_house_set.shape[0]):
        distance = k_nearest_neighbor_argsort(query_house_set[i], set_of_houses, k)
        predictions_dict[i] = np.mean(output_train[distance])
    return predictions_dict

In [112]:
k_prediction_dict

{0: 881300.0,
 1: 431860.0,
 2: 460595.0,
 3: 430200.0,
 4: 766750.0,
 5: 667420.0,
 6: 350032.0,
 7: 512800.7,
 8: 484000.0,
 9: 457235.0}

In [99]:
k_prediction_dict = final_k_nearest_neighbor(10, features_train, output_train, features_test[:10])

**Making predictions for the first 10 houses in the test set using k-nearest neighbors with `k=10`.** 

1. What is the index of the house in this query set that has the lowest predicted value? 
2. What is the predicted value of this house?

In [111]:
min_index = min(k_prediction_dict.items(), key = lambda x : x[1])[0]
print 'Minimum value is of House '+str(min_index)+' having price of %0.0f' % k_prediction_dict[min_index]

Minimum value is of House 6 having price of 350032


## Choosing the best value of k using a validation set

There remains a question of choosing the value of k to use in making predictions. Here, we use a validation set to choose this value. Writing a loop that does the following:

* For `k` in [1, 2, ..., 15]:
    * Makes predictions for each house in the VALIDATION set using the k-nearest neighbors from the TRAINING set.
    * Computes the RSS for these predictions on the VALIDATION set
    * Stores the RSS computed above in `rss_all`
* Report which `k` produced the lowest RSS on VALIDATION set.

# Computing optimal value of K

In [128]:
def finding_optimal_k(k_values, features_train, output_train, output_valid, features_valid):
    rss_k = {}
    for k in k_values:
        predictions = list(final_k_nearest_neighbor(k, features_train, output_train, features_valid).values())
        errors = predictions - output_valid
        rss = sum(errors**2)
        rss_k[k] = rss
    return rss_k

In [129]:
dict_rss = finding_optimal_k(range(1, 16), features_train, output_train, output_valid, features_valid)

In [130]:
dict_rss

{1: 105453830251561.0,
 2: 83445073504025.5,
 3: 72692096019202.8,
 4: 71946721652091.69,
 5: 69846517419718.586,
 6: 68899544353181.09,
 7: 68341973450051.055,
 8: 67361678735491.5,
 9: 68372727958976.336,
 10: 69335048668556.7,
 11: 69523855215598.875,
 12: 69049969587246.45,
 13: 70011254508263.625,
 14: 70908698869034.44,
 15: 71106928385945.36}

# Computing K value having minimum RSS

In [131]:
min(dict_rss.items(), key = lambda x : x[1])

(8, 67361678735491.5)

# Visualizing the performance as a function of `k`

##  Plotting the RSS on the VALIDATION set for each considered `k` value:

In [137]:
import matplotlib.pyplot as plt
%matplotlib inline
rss_all = list(dict_rss.values())
kvals = range(1, 16)
plt.plot(kvals, rss_all,'bo-')
plt.show()

In [136]:
test_predictions = list(final_k_nearest_neighbor(8, features_train, output_train, features_test).values())
test_errors = test_predictions - output_test
test_rss = sum(test_errors**2)
print 'RSS on test set is: %0.2e'%test_rss

RSS on test set is: 1.33e+14


# Good Luck !!!